In [ ]:
%run NB_Credentials

In [40]:
import pandas as pd
from datetime import datetime, timedelta
import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 42, Finished, Available, Finished)

In [41]:
# Iniciar tiempo de ejecución
start_time = time.time()

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 43, Finished, Available, Finished)

In [42]:
# Parámetros
TABLA_DESTINO = "dim_date"

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 44, Finished, Available, Finished)

In [43]:
# Definir el rango de fechas
START_DATE = "2024-01-01"  # Fecha inicial del calendario
END_DATE = "2025-12-31"  # Fecha final del calendario

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 45, Finished, Available, Finished)

In [44]:
# Convertir las fechas a objetos datetime
start_date = datetime.strptime(START_DATE, "%Y-%m-%d")
end_date = datetime.strptime(END_DATE, "%Y-%m-%d")

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 46, Finished, Available, Finished)

In [45]:
# Definir el esquema de la tabla calendario
schema = StructType([
    StructField("date", DateType(), False),
    StructField("year", IntegerType(), False),
    StructField("month", IntegerType(), False),
    StructField("month_name", StringType(), False), 
    StructField("day", IntegerType(), False),
    StructField("weekday", IntegerType(), False),
    StructField("weekday_name", StringType(), False),  
    StructField("week_of_year", IntegerType(), False),
    StructField("quarter", IntegerType(), False),
    StructField("is_weekend", IntegerType(), False)
])

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 47, Finished, Available, Finished)

In [46]:
# Lista donde almacenaremos los datos
date_data = []

# Inicializar fecha actual
current_date = start_date

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 48, Finished, Available, Finished)

In [47]:
# Generar las fechas dentro del rango
while current_date <= end_date:
    date_data.append({
        "date": current_date.date(),  # Fecha completa
        "year": current_date.year,  # Año
        "month": current_date.month,  # Mes en número
        "month_name": current_date.strftime("%B"),  # Nombre del mes
        "day": current_date.day,  # Día del mes
        "weekday": current_date.weekday(),  # Número del día de la semana (0=Lunes, 6=Domingo)
        "weekday_name": current_date.strftime("%A"),  # Nombre del día de la semana
        "week_of_year": current_date.isocalendar()[1],  # Número de semana en el año
        "quarter": (current_date.month - 1) // 3 + 1,  # Trimestre (Q1, Q2, Q3, Q4)
        "is_weekend": 1 if current_date.weekday() in [5, 6] else 0  # Es fin de semana (1=Sí, 0=No)
    })

    # Incrementar la fecha en 1 día
    current_date += timedelta(days=1)

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 49, Finished, Available, Finished)

In [48]:
# Crear DataFrame de Spark con el esquema definido
df_calendar = spark.createDataFrame(date_data, schema=schema)

# Mostrar la tabla antes de guardarla
display(df_calendar)

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 50, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 68a731b1-1871-483d-bc9d-ee184db6e935)

In [49]:
# Guardar en el Lakehouse en formato Delta
df_calendar.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{SILVER_PATH}/{TABLA_DESTINO}")

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 51, Finished, Available, Finished)

In [50]:
# Calcular tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"⏳ Tiempo de ejecución: {execution_time:.2f} segundos")

StatementMeta(, b2589227-aa30-46f2-950c-3a1132b336dd, 52, Finished, Available, Finished)

⏳ Tiempo de ejecución: 15.30 segundos
